In [1]:
# Cell 0: bootstrap Django using settings at ../processor/settings.py
from pathlib import Path
import sys, os

# Ensure the parent directory (which contains the `processor` package) is on sys.path
proj_parent = Path('..').resolve()
if str(proj_parent) not in sys.path:
    sys.path.insert(0, str(proj_parent))

# Point Django to the settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'processor.settings')

# Import and initialize Django
try:
    import django
except ImportError as e:
    raise ImportError("Django is not installed. Install it with: pip install django") from e

django.setup()

# Optional: verify setup
print("Django", django.get_version(), "configured with", os.environ['DJANGO_SETTINGS_MODULE'])

Django 5.2.4 configured with processor.settings


My Data

In [19]:
from ai.models import DetectionProcess
from record.models import Record
from api.utils import get_counter_auto_detection_results
from asgiref.sync import sync_to_async
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from pathlib import Path
import pandas as pd

records = ["252", "253", "256", "257", "260", "261", "262"]

for record_id in records:
    # --- Retrieve record and metadata ---
    try:
        record = await sync_to_async(Record.objects.get)(id=record_id)
    except Record.DoesNotExist:
        record = None
        print(f"No Record found with id={record_id}")

    if record:
        print(f"Record found: id={record.id}, name={getattr(record, 'name', '')}")

        start_time = getattr(record, "start_time", None)
        duration = getattr(record, "duration", None)
        direction = getattr(record, "direction", None)
        ip = getattr(record, "camera_url", None)
        if "192.168.30.44" in ip:
            intersection_name = "Maple Grove"
        if "192.168.30.76" in ip:
            intersection_name = "Fernbrook"
        # Normalize direction
        direction_map = {
            "east": "East Bound",
            "west": "West Bound",
            "north": "North Bound",
            "south": "South Bound",
        }
        direction = direction_map.get(direction, "Unknown")

        # Compute end time
        end_time = None
        try:
            if start_time and duration is not None:
                if isinstance(duration, timedelta):
                    mins = duration.total_seconds() / 60
                else:
                    mins = float(duration)
                end_time = start_time + timedelta(minutes=mins)
        except Exception:
            pass

        # Ensure timezone (America/Chicago)
        CHI = ZoneInfo("America/Chicago")

        def to_chicago(dt):
            if dt is None:
                return None
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=ZoneInfo("UTC"))
            return dt.astimezone(CHI)

        start_time = to_chicago(start_time)
        end_time = to_chicago(end_time)

        print(f"start_time={start_time}, end_time={end_time}, direction={direction}")

    # --- Get detection process ---
    try:
        detection_process = await sync_to_async(DetectionProcess.objects.get)(
            record_id=record_id, done=True
        )
    except DetectionProcess.DoesNotExist:
        detection_process = None
        print(f"No DetectionProcess found for record_id={record_id}")

    # --- Process results ---
    if detection_process:
        results = await sync_to_async(get_counter_auto_detection_results)(
            record_id, detection_process.version, detection_process.divide_time
        )

        cls_map = {2: "car", 3: "motorcycle", 5: "bus", 7: "truck"}
        data = results[0]

        records = []
        for movement, records_dict in data.items():
            for sec, vals in records_dict.items():
                try:
                    count, ids, cls_list = vals
                except Exception:
                    continue
                cls_num = cls_list[0] if cls_list else None
                cls_label = cls_map.get(cls_num, "other")
                timestamp = start_time + timedelta(seconds=float(sec))
                records.append(
                    {
                        "movement": movement,
                        "timestamp": timestamp,
                        "count": int(count),
                        "cls": cls_label,
                    }
                )

        df = pd.DataFrame(records)
        if df.empty:
            print("⚠️ No records found.")
        else:
            # --- Filter between 6 AM and 6 PM ---
            mask = (df["timestamp"].dt.time >= datetime.strptime("06:00", "%H:%M").time()) & \
                (df["timestamp"].dt.time <= datetime.strptime("18:00", "%H:%M").time())
            df = df.loc[mask].copy()

            # --- 15-minute bins ---
            df["time_bin"] = df["timestamp"].dt.floor("15min")

            # --- STEP 1: total number of vehicles in each 15-min bin per movement ---
            bin_summary = (
                df.groupby(["time_bin", "movement"])["count"]
                .sum()
                .unstack(fill_value=0)
                .sort_index()
            )

            # --- STEP 2: totals of cars and trucks across all time per movement ---
            car_totals = (
                df.loc[df["cls"] == "car"]
                .groupby("movement")["count"]
                .sum()
                .reindex(bin_summary.columns, fill_value=0)
            )
            truck_totals = (
                df.loc[df["cls"] == "truck"]
                .groupby("movement")["count"]
                .sum()
                .reindex(bin_summary.columns, fill_value=0)
            )

            # --- STEP 3: append car/truck totals as extra rows ---
            final_summary = pd.concat(
                [bin_summary,
                pd.DataFrame([car_totals, truck_totals], index=["car", "truck"])],
                sort=False
            )

            # --- STEP 4: wrap columns under top-level header = direction ---
            final_summary.columns = pd.MultiIndex.from_product([[direction], final_summary.columns])

            # Ensure index datetimes are timezone-unaware for Excel (Excel does not support tz-aware datetimes).
            # The index may contain mixed types (datetimes plus row labels like "car"/"truck"),
            # so convert only Timestamp entries and leave others unchanged.
            new_index = []
            for v in final_summary.index:
                if isinstance(v, pd.Timestamp):
                    if v.tzinfo is not None:
                        # remove tzinfo while preserving the wall-clock time
                        new_index.append(v.to_pydatetime().replace(tzinfo=None))
                    else:
                        new_index.append(v.to_pydatetime())
                else:
                    new_index.append(v)
            try:
                final_summary.index = pd.Index(new_index)
            except Exception:
                # fallback: convert everything to string if assigning the new index fails
                final_summary.index = pd.Index([str(x) for x in new_index])

            # --- STEP 5: save to Excel ---
            # build a safe filename: <intersection_name>_<direction>_<start>-<end>.xlsx
            def _safe(s):
                return "".join(c if c.isalnum() or c in ("_", "-") else "_" for c in str(s)).strip().replace("__", "_")

            if start_time and end_time:
                # ensure times are in Chicago and use filesystem-safe format
                st = start_time.astimezone(CHI) if getattr(start_time, "tzinfo", None) else start_time
                et = end_time.astimezone(CHI) if getattr(end_time, "tzinfo", None) else end_time
                st_str = st.strftime("%Y%m%d_%H_%M")
                et_str = et.strftime("%Y%m%d_%H_%M")
                base = f"{intersection_name or record_id}_{direction or ''}_{st_str}-{et_str}"
            else:
                base = f"{intersection_name or record_id}_{direction or ''}"

            base = _safe(base)
            output_file = (f"{base}.xlsx") if 'proj_parent' in globals() else Path(f"{base}.xlsx")
            final_summary.to_excel(output_file)

Record found: id=252, name=
start_time=2025-11-04 08:57:54.230000-06:00, end_time=2025-11-04 11:57:54.230000-06:00, direction=West Bound
Record found: id=253, name=
start_time=2025-11-04 08:57:54.242000-06:00, end_time=2025-11-04 11:57:54.242000-06:00, direction=East Bound
Record found: id=256, name=
start_time=2025-11-04 11:58:25.734000-06:00, end_time=2025-11-04 17:58:25.734000-06:00, direction=East Bound
Record found: id=257, name=
start_time=2025-11-04 11:58:25.745000-06:00, end_time=2025-11-04 17:58:25.745000-06:00, direction=West Bound
Record found: id=260, name=
start_time=2025-11-04 06:00:38.447000-06:00, end_time=2025-11-04 07:00:38.447000-06:00, direction=East Bound
Record found: id=261, name=
start_time=2025-11-04 06:00:38.461000-06:00, end_time=2025-11-04 07:00:38.461000-06:00, direction=North Bound
Record found: id=262, name=
start_time=2025-11-04 06:00:38.473000-06:00, end_time=2025-11-04 07:00:38.473000-06:00, direction=West Bound


ISS Data

In [14]:
from datetime import datetime, timedelta
import requests
from datetime import datetime
from zoneinfo import ZoneInfo
CHI = ZoneInfo("America/Chicago")
maple_grove_ip = "192.168.30.44"
fernbrooke_ip = "192.168.30.76"
is_maple_grove = False
camera_id = 3
# create UTC zoneinfo from the existing CHI object and convert parsed CHI times to UTC
UTZ = CHI.__class__("UTC")
start_time = datetime.strptime("2025-11-04 06:00:00AM", "%Y-%m-%d %I:%M:%S%p").replace(tzinfo=CHI).astimezone(UTZ)
end_time = datetime.strptime("2025-11-04 06:00:00PM", "%Y-%m-%d %I:%M:%S%p").replace(tzinfo=CHI).astimezone(UTZ)


if is_maple_grove:
    ip = maple_grove_ip
else:
    ip = fernbrooke_ip

if is_maple_grove:
    if camera_id == 1:
        direction = "East Bound"
    elif camera_id == 2:
        direction = "South Bound"
    elif camera_id == 3:
        direction = "West Bound"
    elif camera_id == 4:
        direction = "North Bound"
else:
    if camera_id == 1:
        direction = "North Bound"
    elif camera_id == 2:
        direction = "West Bound"
    elif camera_id == 3:
        direction = "East Bound"

url = f"http://{ip}/api/v1/cameras/{camera_id}/bin-statistics"
params = {
    "start-time": start_time.strftime("%Y-%m-%dT%H:%M:%S"),
    "end-time": end_time.strftime("%Y-%m-%dT%H:%M:%S")
}
try:
    response = requests.get(url, params=params)
    response.raise_for_status() 
    data = response.json()
except requests.RequestException as e:
    print(f"Error fetching bin-statistics: {e}")
    None


import pandas as pd
from pathlib import Path
# Process ISS data into similar format as detection results
def process_iss_data(data, direction, start_time):
    if not data or 'statistics' not in data:
        print("⚠️ No statistics found in data")
        return None

    # Create records list from statistics
    records = []
    for stat in data['statistics']:
        # Convert UTC time to timestamp
        timestamp = datetime.fromisoformat(stat['time'].replace('Z', '+00:00'))
        timestamp = timestamp.astimezone(CHI)  # Convert to Chicago time
        
        # Map movements based on counts
        movements = {
            'through': stat.get('throughCount', 0),
            'left_turn': stat.get('leftTurnCount', 0),
            'right_turn': stat.get('rightTurnCount', 0)
        }
        
        # Create a record for each movement type that has counts
        for movement, count in movements.items():
            if count > 0:
                records.append({
                    'movement': movement,
                    'timestamp': timestamp,
                    'count': count,
                    'cls': 'vehicle'  # ISS doesn't provide vehicle classification
                })

    # Create DataFrame
    df = pd.DataFrame(records)
    if df.empty:
        print("⚠️ No records found in statistics")
        return None

    # --- Filter between 6 AM and 6 PM ---
    mask = (df['timestamp'].dt.time >= datetime.strptime('06:00', '%H:%M').time()) & \
           (df['timestamp'].dt.time <= datetime.strptime('18:00', '%H:%M').time())
    df = df.loc[mask].copy()

    # --- 15-minute bins ---
    df['time_bin'] = df['timestamp'].dt.floor('15min')

    # --- STEP 1: total vehicles in each 15-min bin per movement ---
    bin_summary = (
        df.groupby(['time_bin', 'movement'])['count']
        .sum()
        .unstack(fill_value=0)
        .sort_index()
    )

    # --- STEP 2: total counts (no car/truck split available in ISS data) ---
    total_counts = df.groupby('movement')['count'].sum().reindex(bin_summary.columns, fill_value=0)

    # --- STEP 3: append totals as extra row ---
    final_summary = pd.concat(
        [bin_summary,
         pd.DataFrame([total_counts], index=['total'])],
        sort=False
    )

    # --- STEP 4: wrap columns under top-level header = direction ---
    final_summary.columns = pd.MultiIndex.from_product([[direction], final_summary.columns])

    # Convert timezone-aware timestamps to naive for Excel compatibility
    new_index = []
    for v in final_summary.index:
        if isinstance(v, pd.Timestamp):
            if v.tzinfo is not None:
                new_index.append(v.to_pydatetime().replace(tzinfo=None))
            else:
                new_index.append(v.to_pydatetime())
        else:
            new_index.append(v)
    
    try:
        final_summary.index = pd.Index(new_index)
    except Exception:
        final_summary.index = pd.Index([str(x) for x in new_index])

    return final_summary

# Process and save the data
if data:
    intersection_name = "Maple Grove" if is_maple_grove else "Fernbrook"
    final_summary = process_iss_data(data, direction, start_time)
    
    if final_summary is not None:
        # Save to Excel with formatted filename
        def _safe(s):
            return "".join(c if c.isalnum() or c in ("_", "-") else "_" for c in str(s)).strip().replace("__", "_")
        
        st_str = start_time.astimezone(CHI).strftime("%Y%m%d_%H_%M")
        et_str = end_time.astimezone(CHI).strftime("%Y%m%d_%H_%M")
        base = f"{intersection_name}_{direction}_{st_str}-{et_str}"
        base = _safe(base)
        output_file = Path(f"{base}.xlsx")
        final_summary.to_excel(output_file)
        print(f"Saved results to {output_file}")
    else:
        print("⚠️ Could not create summary from ISS data")
else:
    print("⚠️ No data received from ISS API")


Saved results to Fernbrook_East_Bound_20251104_06_00-20251104_18_00.xlsx


Saved results to Maple_Grove_East_Bound_20251104_06_00-20251104_18_00.xlsx
